In [18]:
import os
import pandas as pd
import pickle
import ast
import sys
import re

In [19]:
OUT_DIR="/media/SSD/paper_data/"

In [32]:
sensor_tag="all"
seeds = [0, 1, 2, 3, 4]
preprocessings = ["none"]#, "CWT-H", "CWT-V"]
models = ['resnet']#["LSTM", "CNN", "CNN_small", "LSTM+CNN", "CNN+LSTM", "MLP", "resnet"]
datasets = ["IN/nops", "OUT1/nops", "OUT1/random", "OUT2/nops"]

columns = ['dataset', 'size', 'model', 'preprocessing', 'seed', 'accuracy']
accuracies = pd.DataFrame(columns=columns)

results_idx = 0

for seed in seeds:
    for preprocessing in preprocessings:
        for model in models:
            for dataset in datasets:
                for i in range(1000, 10000, 1000):

                    model_small=model.replace("_", "-").replace("+", "-").replace("/", "-").lower()
                    dataset_small=dataset.replace("_", "-").replace("+", "-").replace("/", "-").lower()
                    preprocessing_small=preprocessing.replace("_", "-").replace("+", "-").replace("/", "-").lower()

                    acc_file=OUT_DIR+"/sakura/Exp-"+dataset+"/results/"+model+"_p"+preprocessing+"_"+str(i)+"_s"+str(seed)+"-sensors-"+sensor_tag+"_1fold/out/record/LSTM+CNN_record.txt"

                    # Check if job was completed
                    if(os.path.exists(acc_file)!=True):
                        print("Results for "+acc_file+" not yet ready!")
                        accuracies.loc[results_idx] = ["Exp-"+dataset_small, i, model, preprocessing, seed, "-"]
                        results_idx = results_idx+1
                        continue

                    # Check if data was stored safetly
                    num_lines = sum(1 for line in open(acc_file))
                    if(num_lines < 1):
                        print("Missing data for "+acc_file+" not yet ready!")
                        accuracies.loc[results_idx] = ["Exp-"+dataset_small, i, model, preprocessing, seed, "-"]
                        results_idx = results_idx+1
                        continue
                    #print(out_path)

                    file1 = open(acc_file, 'r')
                    Lines = file1.readlines()

                    for count, line in enumerate(Lines):
                        #print("Line{}: {}".format(count, line.strip()))
                        if(count==4):
                            accuracy = re.findall("\d+\.\d+", line.strip())[0]
                            #row = pd.DataFrame(['Exp-IN', 'LSTM+CNN', 'none', 0, count, kf_accuracy])
                            #accuracies = pd.concat([accuracies, row], axis=0, ignore_index=True)
                            accuracies.loc[results_idx] = ["Exp-"+dataset_small, i, model, preprocessing, seed, float(accuracy)]
                            results_idx = results_idx+1

                    file1.close()
                if(seed == 0):
                    acc_file=OUT_DIR+"/sakura/Exp-"+dataset+"/results/"+model+"_p"+preprocessing+"_"+str(10000)+"_s"+str(seed)+"-sensors-"+sensor_tag+"_10fold/out/kfold_results.txt"
                    
                    # Check if job was completed
                    if(os.path.exists(acc_file)!=True):
                        print("Results for "+"Exp-"+dataset+"/results/"+model+"_p"+preprocessing+"_"+str(10000)+"_s"+str(seed)+"-sensors-"+sensor_tag+"_10fold"+" not yet ready!")
                        for kfold_tmp in range(5):
                            accuracies.loc[results_idx] = ["Exp-"+dataset_small, 10000, model, preprocessing, kfold_tmp, "-"]
                            results_idx = results_idx+1
                        continue
                    
                    # Check if data was stored safetly
                    num_lines = sum(1 for line in open(acc_file))
                    if(num_lines <= 11):
                        print("Missing data for "+"Exp-"+dataset+"/results/"+model+"_p"+preprocessing+"_"+str(10000)+"_s"+str(seed)+"-sensors-"+sensor_tag+"_10fold")
                        for kfold_tmp in range(10):
                            accuracies.loc[results_idx] = ["Exp-"+dataset_small, 10000, model, preprocessing, kfold_tmp, "-"]
                            results_idx = results_idx+1
                        continue
                    #print(out_path)
                    
                    file1 = open(acc_file, 'r')
                    Lines = file1.readlines()
    
                    for count, line in enumerate(Lines):
                        #print("Line{}: {}".format(count, line.strip()))
                        if(count>=1 and count <=5):
                            acc_evol=OUT_DIR+"/sakura/Exp-"+dataset+"/results/"+model+"_p"+preprocessing+"_"+str(10000)+"_s"+str(seed)+"-sensors-"+sensor_tag+"_10fold/out/log/LSTM+CNN_kf"+str(count-1)+".txt"
                            kf_accuracy = re.findall("\d+\.\d+", line.strip())[1]
                            accuracies.loc[results_idx] = ["Exp-"+dataset_small, 10000, model, preprocessing, count-1, float(kf_accuracy)]
                            results_idx = results_idx+1
                    
                    file1.close()

In [34]:
accuracies.to_csv(OUT_DIR+"/sakura/accuracies/accuracies_dataset_size_all.csv", index=False)

In [33]:
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    print(accuracies)

             dataset   size   model preprocessing  seed  accuracy
0        Exp-in-nops   1000  resnet          none     0    79.405
1        Exp-in-nops   2000  resnet          none     0    81.905
2        Exp-in-nops   3000  resnet          none     0    83.234
3        Exp-in-nops   4000  resnet          none     0    84.750
4        Exp-in-nops   5000  resnet          none     0    84.989
5        Exp-in-nops   6000  resnet          none     0    84.959
6        Exp-in-nops   7000  resnet          none     0    86.143
7        Exp-in-nops   8000  resnet          none     0    86.382
8        Exp-in-nops   9000  resnet          none     0    86.372
9        Exp-in-nops  10000  resnet          none     0    86.338
10       Exp-in-nops  10000  resnet          none     1    86.665
11       Exp-in-nops  10000  resnet          none     2    86.519
12       Exp-in-nops  10000  resnet          none     3    86.297
13       Exp-in-nops  10000  resnet          none     4    86.416
14     Exp